In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/24 20:45:29 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.24.22 instead (on interface eth0)
22/08/24 20:45:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/24 20:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/24 20:45:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/24 20:45:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [4]:
# converts the booleans into 0 and 1's
# convert the location ID's into strings for one hot encoding

yellow = yellow.withColumn('pickup_hour', hour("tpep_pickup_datetime"))

yellow = yellow.withColumn('dropoff_hour', hour("tpep_dropoff_datetime"))

yellow = yellow.withColumn('is_weekend', F.when(yellow.is_weekend == 'false', 0).otherwise(1))

yellow = yellow.withColumn('Win', F.when(yellow.Win == 'No', 0).otherwise(1))

yellow = yellow.withColumn('PULocationID', F.col('PULocationID').cast('STRING'))

yellow = yellow.withColumn('DOLocationID', F.col('PULocationID').cast('STRING'))

In [5]:
# code to one hot encode the categorical attributes

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline


categorical = ['PULocationID', 'DOLocationID', 'Start(ET)']

indexers = []

for column in categorical:
    indexers.append(StringIndexer(inputCol=column, outputCol=column+"_NUMERIC").fit(yellow))
    
pipeline = Pipeline(stages=indexers)
yellow_indexed = pipeline.fit(yellow).transform(yellow)
yellow_indexed.limit(10)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,tolls_amount,total_amount,is_weekend,trip_length,Date,Start(ET),Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour,PULocationID_NUMERIC,DOLocationID_NUMERIC,Start(ET)_NUMERIC
2018-12-25 00:39:10,2018-12-25 01:04:33,2.56,162,162,16.5,4.58,0.0,22.88,0,25.3833,2018-12-25,12:00,19812,0,-14,0,1,3.0,3.0,1.0
2018-12-25 09:08:51,2018-12-25 09:13:27,0.9,163,163,5.0,1.45,0.0,7.25,0,4.6,2018-12-25,12:00,19812,0,-14,9,9,14.0,14.0,1.0
2018-12-25 10:29:03,2018-12-25 10:31:40,0.63,100,100,4.0,0.96,0.0,5.76,0,2.6167,2018-12-25,12:00,19812,0,-14,10,10,23.0,23.0,1.0
2018-12-25 15:52:23,2018-12-25 16:07:03,2.13,162,162,11.5,1.0,0.0,13.3,0,14.6667,2018-12-25,12:00,19812,0,-14,15,16,3.0,3.0,1.0
2018-12-25 00:31:15,2018-12-25 00:42:04,2.88,230,230,11.0,2.36,0.0,14.16,0,10.8167,2018-12-25,12:00,19812,0,-14,0,0,10.0,10.0,1.0
2018-12-25 00:51:53,2018-12-25 00:58:03,0.98,161,161,6.0,1.36,0.0,8.16,0,6.1667,2018-12-25,12:00,19812,0,-14,0,0,2.0,2.0,1.0
2018-12-25 03:53:56,2018-12-25 04:06:21,1.64,246,246,9.5,0.0,0.0,10.3,0,12.4167,2018-12-25,12:00,19812,0,-14,3,4,26.0,26.0,1.0
2018-12-25 17:03:49,2018-12-25 17:07:48,0.83,237,237,5.0,2.0,0.0,7.8,0,3.9833,2018-12-25,12:00,19812,0,-14,17,17,0.0,0.0,1.0
2018-12-25 17:38:54,2018-12-25 17:42:43,1.1,237,237,5.5,1.58,0.0,7.88,0,3.8167,2018-12-25,12:00,19812,0,-14,17,17,0.0,0.0,1.0
2018-12-25 19:48:14,2018-12-25 19:51:23,0.71,114,114,4.5,1.06,0.0,6.36,0,3.15,2018-12-25,12:00,19812,0,-14,19,19,28.0,28.0,1.0


In [6]:
# take a sample of the indexed dataframe

SAMPLE_SIZE = 0.10

yellow_indexed_sample = yellow_indexed.sample(SAMPLE_SIZE, seed=0)

sample_pandas = yellow_indexed_sample.toPandas()

In [7]:
# standardized the x columns code from tutorial 2


from scipy.stats import zscore

x_cols = ['fare_amount','tolls_amount', 'total_amount', 'is_weekend', 'trip_length', 
          'Attendance', 'Win', 'margin_victory/loss','pickup_hour', 'dropoff_hour', 
          'PULocationID_NUMERIC', 'DOLocationID_NUMERIC', 'Start(ET)_NUMERIC']

y_cols = ['tip_amount']

df_standard = sample_pandas[x_cols].astype(float).apply(zscore)

# format output to 4 decimal places
pd.options.display.float_format = '{:,.4f}'.format
df_standard.describe().loc[['mean','std']]



,fare_amount,tolls_amount,total_amount,is_weekend,trip_length,Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour,PULocationID_NUMERIC,DOLocationID_NUMERIC,Start(ET)_NUMERIC
mean,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [8]:
# fit the lasso model code from tutorial 2

import numpy as np
from glmnet import ElasticNet

elastic_net_model_lasso = ElasticNet(alpha=1) 
elastic_net_model_lasso.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    sample_pandas[y_cols].values.flatten()
)

ElasticNet()

In [9]:
# fit the ridge model code from tutorial 2

elastic_net_model_ridge = ElasticNet(alpha=0) 
elastic_net_model_ridge.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    sample_pandas[y_cols].values.flatten()
)

ElasticNet(alpha=0)

In [10]:
best_lambda_lasso = elastic_net_model_lasso.lambda_best_[0]
best_lambda_ridge = elastic_net_model_ridge.lambda_best_[0]

print(f'Best lambda value for LASSO:', best_lambda_lasso)
print(f'Best lambda value for Ridge:', best_lambda_ridge)


Best lambda value for LASSO: 0.005125111439378727
Best lambda value for Ridge: 0.09382737702850333


In [11]:
# drop the tpep pickup and dropoff columns, and categorical columns which have not been indexed

yellow_indexed = yellow_indexed.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Start(ET)',
                                    'PULocationID', 'DOLocationID')

In [12]:
# use all 2018 data as a train set

# then use the first game of 2019 as the test set

yellow_train = yellow_indexed.where((F.col('Date') < '2019-01-11'))

yellow_test = yellow_indexed.where((F.col('Date') == '2019-01-11'))


In [13]:
# drop Date columns for both train and test

yellow_train = yellow_train.drop('Date')

yellow_test = yellow_test.drop('Date')

In [14]:
# create a vector for the train dataframe ridge model

from pyspark.ml.feature import VectorAssembler

features = 'features'

# use all featuresn
input_cols_full = ['fare_amount','tolls_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length', 'Win',
                    'PULocationID_NUMERIC', 'DOLocationID_NUMERIC', 'pickup_hour', 'dropoff_hour','Attendance', 
                    'margin_victory/loss', 'Start(ET)_NUMERIC']

vectorAssembler_full = VectorAssembler(
    inputCols = input_cols_full, 
    outputCol = features)


v_yellow_train_full = vectorAssembler_full.transform(yellow_train.dropna('any'))

v_yellow_train_full = v_yellow_train_full.select(['features', 'tip_amount'])

v_yellow_train_full.show(3)

22/08/24 20:48:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,0.0,2.56,22...|      4.58|
|[5.0,0.0,0.9,7.25...|      1.45|
|[4.0,0.0,0.63,5.7...|      0.96|
+--------------------+----------+
only showing top 3 rows



In [15]:
v_yellow_test_full = vectorAssembler_full.transform(yellow_test.dropna('any'))

v_yellow_test_full = v_yellow_test_full.select(['features', 'tip_amount'])

v_yellow_test_full.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,0.0,2.88,18...|       3.7|
|(14,[0,2,3,5,7,8,...|      2.26|
|(14,[0,2,3,5,7,8,...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [16]:
# LassoM stands for LASSO model

from pyspark.ml.regression import LinearRegression

LassoM = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_lasso, elasticNetParam = 1).fit(v_yellow_train_full)

22/08/24 20:48:26 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/24 20:48:26 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [17]:
#coefficients for LASSO regression

pd.DataFrame(
    data=[LassoM.intercept] + list(LassoM.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

# based on LASSO, Attendance, margin_victory/loss, dropoff_hour, and trip_length are all irrelevant

,coefficient
intercept,-0.7366
fare_amount,-0.8070
tolls_amount,-0.8002
trip_distance,-0.0344
total_amount,0.8428
is_weekend,0.1202
trip_length,0.0000
Win,0.0000
PULocationID_NUMERIC,-0.0002
DOLocationID_NUMERIC,-0.0002


In [18]:
trainingSummary_LassoM = LassoM.summary

print("RMSE: %f" % trainingSummary_LassoM.rootMeanSquaredError)
print("r2: %f" % trainingSummary_LassoM.r2)
print("MSE: %f" % trainingSummary_LassoM.meanSquaredError)
print("MAE: %f" % trainingSummary_LassoM.meanAbsoluteError)


RMSE: 0.334951
r2: 0.932626
MSE: 0.112192
MAE: 0.262370


In [19]:
v_yellow_train_full.describe().show()

+-------+-----------------+
|summary|       tip_amount|
+-------+-----------------+
|  count|          2398168|
|   mean|2.042468496785995|
| stddev|1.290430556720419|
|    min|              0.0|
|    max|            411.0|
+-------+-----------------+



In [20]:
features = 'features'

# features selected after selection through LASSO
input_cols_reduced = ['fare_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length',
                     'PULocationID_NUMERIC', 'DOLocationID_NUMERIC', 'pickup_hour', 'Win', 'Start(ET)_NUMERIC']


vectorAssembler_reduced = VectorAssembler(
    inputCols = input_cols_reduced, 
    outputCol = features)


v_yellow_train_reduced = vectorAssembler_reduced.transform(yellow_train.dropna('any'))

v_yellow_train_reduced = v_yellow_train_reduced.select(['features', 'tip_amount'])

v_yellow_train_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,2.56,22.88,...|      4.58|
|[5.0,0.9,7.25,0.0...|      1.45|
|[4.0,0.63,5.76,0....|      0.96|
+--------------------+----------+
only showing top 3 rows



In [21]:
v_yellow_test_reduced = vectorAssembler_reduced.transform(yellow_test.dropna('any'))

v_yellow_test_reduced = v_yellow_test_reduced.select(['features', 'tip_amount'])

v_yellow_test_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,2.88,18.5,0...|       3.7|
|[10.0,2.73,13.56,...|      2.26|
|[8.0,2.03,11.16,0...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [22]:
lm = LinearRegression(featuresCol='features', labelCol = 'tip_amount').fit(v_yellow_train_reduced)

22/08/24 20:48:53 WARN Instrumentation: [72ba7fb3] regParam is zero, which might cause numerical instability and overfitting.


22/08/24 20:49:04 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/08/24 20:49:04 WARN Instrumentation: [72ba7fb3] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [23]:
pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols_reduced,
    columns=['coefficient']
)

,coefficient
intercept,-0.6029
fare_amount,-0.7550
trip_distance,-0.0578
total_amount,0.7956
is_weekend,0.1193
trip_length,0.0014
PULocationID_NUMERIC,-0.0007
DOLocationID_NUMERIC,-0.0007
pickup_hour,-0.0156
Win,-0.0093


In [24]:
trainingSummary_lm = lm.summary

print("RMSE: %f" % trainingSummary_lm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_lm.r2)
print("MSE: %f" % trainingSummary_lm.meanSquaredError)
print("MAE: %f" % trainingSummary_lm.meanAbsoluteError)


RMSE: 0.440091
r2: 0.883690
MSE: 0.193680
MAE: 0.274930


In [25]:
from pyspark.ml.evaluation import RegressionEvaluator


lm_predictions = lm.transform(v_yellow_test_reduced)
lm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
| 4.017312264494462|       3.7|[13.0,2.88,18.5,0...|
| 2.448578881792878|      2.26|[10.0,2.73,13.56,...|
|2.1264127982613363|      1.86|[8.0,2.03,11.16,0...|
| 2.625039480088274|      2.49|[7.0,1.39,10.79,0...|
|  4.11479352702198|      4.06|[19.0,5.88,24.36,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [26]:
test_result_lm = lm.evaluate(v_yellow_test_reduced)

print("R Squared (R2) on test data = %g" % test_result_lm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_lm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_lm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_lm.meanAbsoluteError)

R Squared (R2) on test data = 0.895011
Root Mean Squared Error (RMSE) on test data = 0.435432
Mean Squared Error (MSE) on test data = 0.189601
Mean Absolute Error (MAE) on test data = 0.283667


In [27]:
# rm stands for ridge model

rm = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_ridge, elasticNetParam = 0).fit(v_yellow_train_full)

In [28]:
pd.DataFrame(
    data=[rm.intercept] + list(rm.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

,coefficient
intercept,-0.0428
fare_amount,-0.1203
tolls_amount,-0.1305
trip_distance,-0.0640
total_amount,0.2762
is_weekend,0.0077
trip_length,-0.0066
Win,-0.0003
PULocationID_NUMERIC,-0.0013
DOLocationID_NUMERIC,-0.0013


In [29]:
trainingSummary_rm = rm.summary

print("RMSE: %f" % trainingSummary_rm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_rm.r2)
print("MSE: %f" % trainingSummary_rm.meanSquaredError)
print("MAE: %f" % trainingSummary_rm.meanAbsoluteError)


RMSE: 0.765345
r2: 0.648241
MSE: 0.585753
MAE: 0.389100


In [30]:
rm_predictions = rm.transform(v_yellow_test_full)
rm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|3.0635689339405525|       3.7|[13.0,0.0,2.88,18...|
| 2.115710779425697|      2.26|(14,[0,2,3,5,7,8,...|
|1.8159809532824283|      1.86|(14,[0,2,3,5,7,8,...|
|1.8793977552018994|      2.49|(14,[0,2,3,5,7,8,...|
|3.8497936092227176|      4.06|(14,[0,2,3,5,7,8,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [31]:
test_result_rm = rm.evaluate(v_yellow_test_full)

print("R Squared (R2) on test data = %g" % test_result_rm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_rm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_rm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_rm.meanAbsoluteError)

R Squared (R2) on test data = 0.623506
Root Mean Squared Error (RMSE) on test data = 0.82457
Mean Squared Error (MSE) on test data = 0.679916
Mean Absolute Error (MAE) on test data = 0.36117
